In [1]:
import pprint
import urllib.parse
import json5
from qwen_agent.agents import Assistant
from qwen_agent.tools.base import BaseTool, register_tool
import os

# 步骤 1（可选）：添加一个名为 `my_image_gen` 的自定义工具。
@register_tool('my_image_gen')
class MyImageGen(BaseTool):
    # `description` 用于告诉智能体该工具的功能。
    description = 'AI 绘画（图像生成）服务，输入文本描述，返回基于文本信息绘制的图像 URL。'
    # `parameters` 告诉智能体该工具有哪些输入参数。
    parameters = [{
        'name': 'prompt',
        'type': 'string',
        'description': '期望的图像内容的详细描述',
        'required': True
    }]

    def call(self, params: str, **kwargs) -> str:
        # `params` 是由 LLM 智能体生成的参数。
        prompt = json5.loads(params)['prompt']
        prompt = urllib.parse.quote(prompt)
        return json5.dumps(
            {'image_url': f'https://image.pollinations.ai/prompt/{prompt}'},
            ensure_ascii=False)


# 步骤 2：配置您所使用的 LLM。
llm_cfg = {
    # 使用 DashScope 提供的模型服务：
    'model': 'qwen-max',
    'model_server': 'dashscope',
    'api_key': os.getenv('DASHSCOPE_API_KEY'),  # 从环境变量获取API Key
    'generate_cfg': {
        'top_p': 0.8
    }
}

llm_cfg = {
    # 使用 DashScope 提供的模型服务：
    'model': 'deepseek-v3',
    'model_server': 'https://dashscope.aliyuncs.com/compatible-mode/v1',
    'api_key': os.getenv('DASHSCOPE_API_KEY'),  # 从环境变量获取API Key
    'generate_cfg': {
        'top_p': 0.8
    }
}

# 步骤 3：创建一个智能体。这里我们以 `Assistant` 智能体为例，它能够使用工具并读取文件。
system_instruction = '''你是一个乐于助人的AI助手。
在收到用户的请求后，你应该：
- 首先绘制一幅图像，得到图像的url，
- 然后运行代码`request.get`以下载该图像的url，
- 最后从给定的文档中选择一个图像操作进行图像处理。
用 `plt.show()` 展示图像。
你总是用中文回复用户。'''
tools = ['my_image_gen', 'code_interpreter']  # `code_interpreter` 是框架自带的工具，用于执行代码。
import os
# 获取文件夹下所有文件
file_dir = os.path.join('./', 'docs')
files = []
if os.path.exists(file_dir):
    # 遍历目录下的所有文件
    for file in os.listdir(file_dir):
        file_path = os.path.join(file_dir, file)
        if os.path.isfile(file_path):  # 确保是文件而不是目录
            files.append(file_path)
print('files=', files)

bot = Assistant(llm=llm_cfg,
                system_message=system_instruction,
                function_list=tools,
                files=files)

# 步骤 4：作为聊天机器人运行智能体。
messages = []  # 这里储存聊天历史。
query = "介绍下雇主责任险"
# 将用户请求添加到聊天历史。
messages.append({'role': 'user', 'content': query})
response = []
current_index = 0
for response in bot.run(messages=messages):
    if current_index == 0:
        # 尝试获取并打印召回的文档内容
        if hasattr(bot, 'retriever') and bot.retriever:
            print("\n===== 召回的文档内容 =====")
            retrieved_docs = bot.retriever.retrieve(query)
            if retrieved_docs:
                for i, doc in enumerate(retrieved_docs):
                    print(f"\n文档片段 {i+1}:")
                    print(f"内容: {doc.page_content}")
                    print(f"元数据: {doc.metadata}")
            else:
                print("没有召回任何文档内容")
            print("===========================\n")
        #break

    current_response = response[0]['content'][current_index:]
    current_index = len(response[0]['content'])
    print(current_response, end='')
# 将机器人的回应添加到聊天历史。
#messages.extend(response)

files= ['./docs\\1-平安商业综合责任保险（亚马逊）.txt', './docs\\2-雇主责任险.txt', './docs\\3-平安企业团体综合意外险.txt', './docs\\4-雇主安心保.txt', './docs\\5-施工保.txt', './docs\\6-财产一切险.txt', './docs\\7-平安装修保.txt', './docs\\平安产险交通出行意外伤害保险（互联网版）产品说明.pdf', './docs\\平安产险交通工具意外伤害保险（互联网版）条款.pdf', './docs\\平安企业团体综合意外险(互联网版)适用条款.pdf', './docs\\平安商业综合责任保险（亚马逊）.pdf', './docs\\平安境内紧急医疗救援服务条款.pdf', './docs\\平安附加疾病身故保险条款.pdf']


2025-06-06 22:13:40,610 - memory.py - 130 - INFO - {"keywords_zh": ["雇主", "责任险", "保险", "介绍", "责任", "雇主责任"], "keywords_en": ["employer", "liability insurance", "insurance", "introduction", "liability", "employer liability"], "text": "介绍下雇主责任险"}
2025-06-06 22:13:41,481 - simple_doc_parser.py - 425 - INFO - Read parsed ./docs\1-平安商业综合责任保险（亚马逊）.txt from cache.
2025-06-06 22:13:41,483 - doc_parser.py - 122 - INFO - Start chunking ./docs\1-平安商业综合责任保险（亚马逊）.txt (1-平安商业综合责任保险（亚马逊）.txt)...
2025-06-06 22:13:41,485 - doc_parser.py - 140 - INFO - Finished chunking ./docs\1-平安商业综合责任保险（亚马逊）.txt (1-平安商业综合责任保险（亚马逊）.txt). Time spent: 0.0 seconds.
2025-06-06 22:13:41,491 - simple_doc_parser.py - 425 - INFO - Read parsed ./docs\2-雇主责任险.txt from cache.
2025-06-06 22:13:41,492 - doc_parser.py - 122 - INFO - Start chunking ./docs\2-雇主责任险.txt (2-雇主责任险.txt)...
2025-06-06 22:13:41,493 - doc_parser.py - 140 - INFO - Finished chunking ./docs\2-雇主责任险.txt (2-雇主责任险.txt). Time spent: 0.0 seconds.
2025-06-06 22:13:41,

雇主责任险是一种专门为企业设计的保险产品，主要目的是转移企业在雇佣员工过程中可能面临的法律责任和经济风险。以下是雇主责任险的详细介绍：

### **1. 保障范围**
雇主责任险主要覆盖以下风险：
- **死亡赔偿金**：员工因工作原因死亡，企业需支付的赔偿。
- **伤残赔偿金**：员工因工作原因伤残，企业需支付的赔偿。
- **医疗费用**：员工因工作原因受伤或患职业病产生的医疗费用。
- **误工费用**：员工因工伤无法工作期间的工资补偿（部分产品不包含）。
- **法律诉讼费**：因工伤纠纷产生的法律费用（部分产品不包含）。

### **2. 适用场景**
- **高风险行业**：如建筑、制造业等。
- **用工风险高**：员工易发生工伤或职业病的行业。
- **法律要求**：部分行业或地区要求企业必须购买雇主责任险。

### **3. 产品亮点**
- **转嫁风险**：将企业对员工的法律责任转移给保险公司。
- **税前列支**：保费可抵扣企业所得税（25%）。
- **灵活定制**：可根据员工工种、行业类型等调整保障方案。
- **减少纠纷**：快速理赔，减少工伤纠纷。

### **4. 与工伤保险的区别**
- **工伤保险**：国家强制，覆盖基本工伤待遇，但企业仍需承担部分费用（如误工费、就业补助金等）。
- **雇主责任险**：商业保险，补充工伤保险的不足，覆盖更全面的赔偿责任。

### **5. 理赔案例**
- **案例1**：员工在上班途中遭遇车祸身亡，保险公司赔付30万元。
- **案例2**：空调安装工人高空坠亡，保险公司赔付95万元。

### **6. 投保建议**
- **高风险企业**：建议优先投保，尤其是建筑、制造业。
- **员工福利**：可作为员工福利，提高员工保障。
- **合规需求**：部分行业需满足安监局要求。

如果需要更具体的保障方案或保费测算，可以提供企业行业、员工人数等信息，我可以进一步协助！

In [1]:
#help(Assistant)
#.add_files()